In [70]:
%%bash
pip install 'great_expectations'

Python(49238) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [71]:
import os
import pandas as pd
import great_expectations as gx
from great_expectations.data_context import FileDataContext

In [ ]:
%%bash
pip install streamlit

In [ ]:
import streamlit as st
import numpy as np
from io import StringIO

In [72]:
st.title("Freddie Mac Dataset Quality Assessment App")

# File upload
uploaded_file = st.file_uploader("Upload a CSV or XLS file", type=["csv", "xls"])

# User selects data type
data_type = st.radio("Select Data Type:", ("Origination Data", "Monthly Performance Data"))

if uploaded_file is not None:
    # Load data into Pandas DataFrame
    if data_type == "Origination Data":
        df = pd.read_csv(uploaded_file)
    else:
        df = pd.read_excel(uploaded_file)

2023-10-05 13:11:07.182 
  command:

    streamlit run /Users/anamikabharali/Documents/UNI/.venv/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


In [73]:
path_to_repo_dir="/Users/anamikabharali/Documents/UNI/BD/streamlit" #local path to csv
path_to_data_dir=f"{path_to_repo_dir}/gx/data" 
expectation_suite_name="GE_Suite" 

In [74]:
context = FileDataContext.create(project_root_dir=path_to_repo_dir)

In [75]:
os.mkdir(path_to_data_dir)

df = pd.read_csv('/Users/anamikabharali/Documents/UNI/BD/streamlit/historical_data_2022Q1.txt', delimiter="|", header = None)

# Save the DataFrame as a CSV file
df.to_csv(f"{path_to_repo_dir}/gx/data/historical_data_2022Q1.csv", index=False)

In [ ]:
dtypes = {'Loan_Sequence_Number','Monthly_Reporting_Period','Current_Actual_UPB','Current_Loan_Delinquency_Status','Loan_Age','Remaining_Months_to_Legal_Maturity','Defect_Settlement_Date','Modification_Flag','Zero_Balance_Code','Zero_Balance_Effective_Date','Current_Interest_Rate','Current_Deferred_UPB','Due_Date_of_Last_Paid_Installment_(DDLPI)','MI_Recoveries','Net_Sales_Proceeds','Non_MI_Recoveries','Expenses','Legal_Costs','Maintenance_and_Preservation_Costs','Taxes_and_Insurance','Miscellaneous_Expenses','Actual_Loss_Calculation','Modification_Cost','Step_Modification_Flag','Deferred_Payment_Plan','Estimated_Loan _to_Value_(ELTV)','Zero_Balance_Removal_UPB','Delinquent_Accrued_Interest','Delinquency_Due_to_Disaster','Borrower_Assistance_Status_Code','Current_Month_Modification_Cost','Interest_Bearing_UPB'
    'Name': str,
    'Age': int,
    'Location': str
}

In [76]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,763,202203,N,205202,47664.0,0,1,P,95,47,625000,56,3.375,R,N,FRM,MI,SF,48000,F22Q10000001,P,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
1,691,202203,N,203702,47664.0,0,1,P,67,25,548000,43,2.750,R,N,FRM,MI,SF,48000,F22Q10000002,C,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
2,678,202203,N,203702,NaN,0,2,P,28,44,208000,28,2.375,R,N,FRM,CO,SF,81200,F22Q10000003,C,180,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,2,N,7
3,802,202203,N,205202,33460.0,0,1,P,75,27,216000,75,3.250,R,N,FRM,WI,SF,54000,F22Q10000004,P,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
4,767,202203,N,205202,NaN,0,1,P,55,44,150000,55,2.875,R,N,FRM,NY,SF,14800,F22Q10000005,N,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7


In [77]:
df.columns=['Loan_Sequence_Number','Monthly_Reporting_Period','Current_Actual_UPB','Current_Loan_Delinquency_Status','Loan_Age','Remaining_Months_to_Legal_Maturity','Defect_Settlement_Date','Modification_Flag','Zero_Balance_Code','Zero_Balance_Effective_Date','Current_Interest_Rate','Current_Deferred_UPB','Due_Date_of_Last_Paid_Installment_(DDLPI)','MI_Recoveries','Net_Sales_Proceeds','Non_MI_Recoveries','Expenses','Legal_Costs','Maintenance_and_Preservation_Costs','Taxes_and_Insurance','Miscellaneous_Expenses','Actual_Loss_Calculation','Modification_Cost','Step_Modification_Flag','Deferred_Payment_Plan','Estimated_Loan _to_Value_(ELTV)','Zero_Balance_Removal_UPB','Delinquent_Accrued_Interest','Delinquency_Due_to_Disaster','Borrower_Assistance_Status_Code','Current_Month_Modification_Cost','Interest_Bearing_UPB']

In [ ]:
df.columns=['Credit_Score', 'First_Payment_Date', 'First_Time_Homebuyer_Flag','Maturity_Date','MSA', 'MI','Number_of_Units','Occupancy_Status','CLTV','DTI','UPB','LTV','Original_Interest_Rate','Channel','PPM','Amortization','Property_State','Property_Type','Postal_Code','Loan_Sequence_Number','Loan_Purpose','Original_Loan_Term','Number_of_Borrowers','Seller_Name','Servicer_Name','Super_Conforming_Flag','HARP','Program_Indicator','HARP_Indicator','Property_Valuation_Method','nterest_Only','Mortgage_Insurance_Cancellation_Indicator']
#for origination data

In [78]:
df.head()

,Loan_Sequence_Number,Monthly_Reporting_Period,Current_Actual_UPB,Current_Loan_Delinquency_Status,Loan_Age,Remaining_Months_to_Legal_Maturity,Defect_Settlement_Date,Modification_Flag,Zero_Balance_Code,Zero_Balance_Effective_Date,Current_Interest_Rate,Current_Deferred_UPB,Due_Date_of_Last_Paid_Installment_(DDLPI),MI_Recoveries,Net_Sales_Proceeds,Non_MI_Recoveries,Expenses,Legal_Costs,Maintenance_and_Preservation_Costs,Taxes_and_Insurance,Miscellaneous_Expenses,Actual_Loss_Calculation,Modification_Cost,Step_Modification_Flag,Deferred_Payment_Plan,Estimated_Loan _to_Value_(ELTV),Zero_Balance_Removal_UPB,Delinquent_Accrued_Interest,Delinquency_Due_to_Disaster,Borrower_Assistance_Status_Code,Current_Month_Modification_Cost,Interest_Bearing_UPB
0,763,202203,N,205202,47664.0,0,1,P,95,47,625000,56,3.375,R,N,FRM,MI,SF,48000,F22Q10000001,P,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
1,691,202203,N,203702,47664.0,0,1,P,67,25,548000,43,2.750,R,N,FRM,MI,SF,48000,F22Q10000002,C,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
2,678,202203,N,203702,NaN,0,2,P,28,44,208000,28,2.375,R,N,FRM,CO,SF,81200,F22Q10000003,C,180,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,2,N,7
3,802,202203,N,205202,33460.0,0,1,P,75,27,216000,75,3.250,R,N,FRM,WI,SF,54000,F22Q10000004,P,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
4,767,202203,N,205202,NaN,0,1,P,55,44,150000,55,2.875,R,N,FRM,NY,SF,14800,F22Q10000005,N,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7


In [ ]:
#df = df.drop('ColumnName', axis=1)
df.head()

In [79]:
# Give your Datasource a name
datasource_name = "Local_FileSystem_Source"
datasource = context.sources.add_pandas_filesystem(name=datasource_name, base_directory=path_to_data_dir)

# Give your first Asset a name
asset_name = "Freddie_Mac2"
batching_regex = r"historical_data_\d{4}Q\d.csv" #change this

asset = datasource.add_csv_asset(name=asset_name,batching_regex=batching_regex)

# Build batch request
batch_request = asset.build_batch_request()

Create expectations suite

In [80]:
data_asset = context.get_datasource(datasource_name).get_asset(asset_name)
batch_request = data_asset.build_batch_request()

In [81]:
context.list_expectation_suite_names()

[]

In [82]:
context. add_or_update_expectation_suite(expectation_suite_name)

{
  "expectation_suite_name": "GE_Suite",
  "ge_cloud_id": null,
  "expectations": [],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.17.19"
  }
}

In [83]:
validator = context.get_validator(batch_request=batch_request, expectation_suite_name=expectation_suite_name)
validator. head ()

Python(49550) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,763,202203,N,205202,47664.0,0,1,P,95,47,625000,56,3.375,R,N,FRM,MI,SF,48000,F22Q10000001,P,360,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
1,691,202203,N,203702,47664.0,0,1,P,67,25,548000,43,2.750,R,N,FRM,MI,SF,48000,F22Q10000002,C,180,2,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
2,678,202203,N,203702,NaN,0,2,P,28,44,208000,28,2.375,R,N,FRM,CO,SF,81200,F22Q10000003,C,180,2,Other sellers,U.S. BANK N.A.,NaN,NaN,9,NaN,2,N,7
3,802,202203,N,205202,33460.0,0,1,P,75,27,216000,75,3.250,R,N,FRM,WI,SF,54000,F22Q10000004,P,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7
4,767,202203,N,205202,NaN,0,1,P,55,44,150000,55,2.875,R,N,FRM,NY,SF,14800,F22Q10000005,N,360,1,Other sellers,Other servicers,NaN,NaN,9,NaN,2,N,7


In [ ]:
validator.expect_column_values_to_not_be_null("VendorID") 
validator.expect_column_values_to_be_in_set ("VendorID", [1, 2]) 
validator.expect_column_values_to_not_be_null("tpep_pickup_datetime")
validator.expect_column_values_to_not_be_null ("tpep_dropoff_datetime") 
validator.expect_column_values_to_be_in_set ("RatecodeID", [1,2,3,4,5,6])
validator.expect_column_values_to_be_in_set ("store_and_fwd_flag", ["Y","N"]) 
validator.expect_column_values_to_be_in_set ("payment_type", [1,2,3,4,5,6])
validator.expect_column_values_to_be_between ("passenger_count", auto=True)